# Install Dependencies

In [ ]:
!pip install fastapi uvicorn pillow torch tensorflow

# Optionally, ins tall localtunnel if you want to expose your local server to the internet
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 911.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  

In [ ]:
!pip install fastapi uvicorn pillow torch tensorflow
!npm install -g localtunnel


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.886s


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.8.1       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.8.1   │
   │               Run npm install -g npm to update!                │
   │                                                                │
   ╰────────────────────────────────────────────────────────────────╯



In [ ]:
!pip install -qU datasets transformers sentence-transformers \
                 pinecone-client==3.1.0 pinecone-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14

# Setup Pinecone

In [ ]:
import os
from pinecone import Pinecone

pc = Pinecone(api_key='2f1689ff-2b91-4d12-8657-8ccd3c15abb6')

index_name = "hybrid-image-search"

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 960}},
 'total_vector_count': 960}

# Setup Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device=device
)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

# Setup BM25

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder()

In [ ]:
import pandas as pd

data = pd.read_csv('/content/rtr_combined.csv')
data.drop_duplicates()
data['title'] = data['title'] + ' ' + data['designer']

In [ ]:
bm25.fit(data['title'])

  0%|          | 0/960 [00:00<?, ?it/s]

# Search Functions

In [ ]:
def hybrid_score_norm(dense, sparse, alpha: float):
    """Hybrid score using a convex combination

    alpha * dense + (1 - alpha) * sparse

    Args:
        dense: Array of floats representing
        sparse: a dict of `indices` and `values`
        alpha: scale between 0 and 1
    """
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    hs = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    return [v * alpha for v in dense], hs

def retrieve_by_filter(query, category=None, top_k=20, alpha=0.5):
  sparse = bm25.encode_queries(query)
  dense = model.encode(query).tolist()

  hdense, hsparse = hybrid_score_norm(dense, sparse, alpha=alpha)

  if category is None:
    result = index.query(
      top_k=top_k,
      vector=hdense,
      sparse_vector=hsparse,
      include_metadata=True
    )
  else:
    result = index.query(
      top_k=top_k,
      vector=hdense,
      sparse_vector=hsparse,
      filter={
          "category": {"$eq": category}
      },
      include_metadata=True
    )

  # return result

  return [d['id'] for d in result['matches']]

In [ ]:
import json

def is_json_serializable(obj):
    try:
        json.dumps(obj)
        return True
    except TypeError:
        return False

result = retrieve_by_filter("red", "Blazers")
print(is_json_serializable(result))
result

True


['311',
 '356',
 '318',
 '348',
 '308',
 '346',
 '323',
 '307',
 '359',
 '334',
 '339',
 '312',
 '320',
 '338',
 '328',
 '340',
 '321',
 '341',
 '352',
 '322']

# Search API

In [ ]:
%%writefile search_api.py
import os
from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import JSONResponse
from pathlib import Path
import sys
import json
import numpy as np
from pydantic import BaseModel
from typing import List, Optional


# Setup project root and import paths
PROJECT_ROOT = Path(__file__).absolute().parents[1].absolute()
sys.path.insert(0, str(PROJECT_ROOT))

app = FastAPI()


# Setup Pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec
import time

pc = Pinecone(api_key='2f1689ff-2b91-4d12-8657-8ccd3c15abb6')

index_name = "hybrid-image-search"

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)


# Setup Sentence Transformer
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device=device
)


# Setup BM25
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder()

import pandas as pd

data = pd.read_csv('/content/rtr_combined.csv')
data.drop_duplicates()
data['title'] = data['title'] + ' ' + data['designer']

bm25.fit(data['title'])


# Setup Functions
def hybrid_score_norm(dense, sparse, alpha: float):
    """Hybrid score using a convex combination

    alpha * dense + (1 - alpha) * sparse

    Args:
        dense: Array of floats representing
        sparse: a dict of `indices` and `values`
        alpha: scale between 0 and 1
    """
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    hs = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    return [v * alpha for v in dense], hs

def retrieve_by_filter(query, category=None, top_k=20, alpha=0.5):
  sparse = bm25.encode_queries(query)
  dense = model.encode(query).tolist()

  hdense, hsparse = hybrid_score_norm(dense, sparse, alpha=alpha)

  if category is None:
    result = index.query(
      top_k=top_k,
      vector=hdense,
      sparse_vector=hsparse,
      include_metadata=True
    )
  else:
    result = index.query(
      top_k=top_k,
      vector=hdense,
      sparse_vector=hsparse,
      filter={
          "category": {"$eq": category}
      },
      include_metadata=True
    )

  return [d['id'] for d in result['matches']]


# GET API
@app.get("/search/")
async def search(
    query: str,
    category: Optional[str] = None,
    top_k: int = Query(20, ge=1),
    alpha: float = Query(0.5, ge=0, le=1)
):
    try:
        results = retrieve_by_filter(
            query=query,
            category=category,
            top_k=top_k,
            alpha=alpha
        )
        return results
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=7879, reload=True)


Overwriting search_api.py


In [ ]:
import threading
import time
import uvicorn
import os
import subprocess

# Ensure no existing processes are using port 7879
!fuser -k 7879/tcp

# Function to run the FastAPI server
def run_uvicorn():
    uvicorn.run("search_api:app", host="0.0.0.0", port=7879)

# Kill any existing Uvicorn processes
os.system("pkill -f 'uvicorn'")

# Start the Uvicorn server in a separate thread
uvicorn_thread = threading.Thread(target=run_uvicorn)
uvicorn_thread.start()

# Wait for the server to start
time.sleep(5)

# Run localtunnel as a subprocess
localtunnel_process = subprocess.Popen(["lt", "--port", "7879"], stdout=subprocess.PIPE)

# Output the localtunnel URL
for line in iter(localtunnel_process.stdout.readline, b''):
    print(line.decode(), end='')


  0%|          | 0/960 [00:00<?, ?it/s]

INFO:     Started server process [498]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7879 (Press CTRL+C to quit)


your url is: https://fresh-cloths-accept.loca.lt
INFO:     34.106.193.242:0 - "GET /search/?query=black&category=Blazers HTTP/1.1" 200 OK
INFO:     34.106.193.242:0 - "GET /search/?query=hitam&category=Blazers HTTP/1.1" 200 OK
